In [1]:
import pandas as pd
import numpy as np

In [39]:
#data = pd.read_csv("../../merged_dataset_ellie.csv")
data = pd.read_csv("../../data/totaldata_encoded_jun11.csv", encoding = "ISO-8859-1")

In [40]:
data.sample(10)

,Unnamed: 0,Unnamed: 0.1,ID,AGEGRP,FIPS_x,YEAR,STATE,COUNTY,STNAME,CTYNAME,...,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%,Unemployment_Rate
11824,15168,15168,200449055,Total,49055,2004,49,55,Utah,Wayne County,...,0.000791,0.489118,0.493075,0.497824,0.502176,0.330431,0.212109,0.257618,0.199842,6.6
10011,13334,13334,200421025,Total,21025,2004,21,25,Kentucky,Breathitt County,...,0.000000,0.498675,0.488939,0.504570,0.495430,0.269373,0.264538,0.293946,0.172142,7.3
8476,11746,11746,200048005,Total,48005,2000,48,5,Texas,Angelina County,...,0.000087,0.421327,0.411992,0.509205,0.490795,0.306554,0.277760,0.248006,0.167680,4.8
6785,10028,10028,200019125,Total,19125,2000,19,125,Iowa,Marion County,...,0.000156,0.492948,0.484804,0.503651,0.496349,0.288411,0.254150,0.255617,0.201822,2.2
5863,6235,6235,201649001,Total,49001,2016,49,1,Utah,Beaver County,...,0.002489,0.460569,0.486390,0.486545,0.513455,0.348266,0.236273,0.221807,0.193654,4.9
9005,12276,12276,200053053,Total,53053,2000,53,53,Washington,Pierce County,...,0.004489,0.405977,0.404554,0.502648,0.497352,0.301831,0.297396,0.264671,0.136101,5.6
5791,6163,6163,201648365,Total,48365,2016,48,365,Texas,Panola County,...,0.000295,0.406712,0.402164,0.504653,0.495347,0.269022,0.239631,0.255842,0.235505,7.2
10650,13993,13993,200430093,Total,30093,2004,30,93,Montana,Silver Bow County,...,0.000568,0.481359,0.475025,0.502420,0.497580,0.258828,0.237319,0.289747,0.214106,5.1
14992,18409,18409,200851061,Total,51061,2008,51,61,Virginia,Fauquier County,...,0.000343,0.445245,0.437323,0.505692,0.494308,0.285487,0.216135,0.329809,0.168569,3.3
10422,13765,13765,200428031,Total,28031,2004,28,31,Mississippi,Covington County,...,0.000051,0.325890,0.315666,0.516328,0.483672,0.308291,0.259207,0.252391,0.180112,5.5


In [41]:
## some cleanup..

data.isnull().sum()

Unnamed: 0           0
Unnamed: 0.1         0
ID                   0
AGEGRP               0
FIPS_x               0
                    ..
TOT_POP_LESS19%      0
TOT_POP_20to39%      0
TOT_POP_40to59%      0
TOT_POP_Above60%     0
Unemployment_Rate    0
Length: 67, dtype: int64

In [8]:
data.dropna(inplace=True)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15437 entries, 0 to 15436
Data columns (total 67 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            15437 non-null  int64  
 1   Unnamed: 0.1          15437 non-null  int64  
 2   ID                    15437 non-null  int64  
 3   AGEGRP                15437 non-null  object 
 4   FIPS_x                15437 non-null  int64  
 5   YEAR                  15437 non-null  int64  
 6   STATE                 15437 non-null  int64  
 7   COUNTY                15437 non-null  int64  
 8   STNAME                15437 non-null  object 
 9   CTYNAME               15437 non-null  object 
 10  AA_FEMALE             15437 non-null  int64  
 11  AA_MALE               15437 non-null  int64  
 12  BA_FEMALE             15437 non-null  int64  
 13  BA_MALE               15437 non-null  int64  
 14  H_FEMALE              15437 non-null  int64  
 15  H_MALE             

In [17]:
data["winning_party_binary"]

0        1
1        1
2        0
3        1
4        1
        ..
15432    0
15433    0
15434    0
15435    0
15436    0
Name: winning_party_binary, Length: 15437, dtype: int64

# cleaning/wrangling...

In [200]:
# data['state'] = data['state'].astype('category').cat.codes
# data['state_po'] = data['state_po'].astype('category').cat.codes
# data['county'] = data['county'].astype('category').cat.codes
# data['winning party'] = data['winning party'].astype('category').cat.codes

In [201]:
# data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15769 entries, 0 to 15787
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  15769 non-null  int64  
 1   state                 15769 non-null  int8   
 2   state_po              15769 non-null  int8   
 3   county                15769 non-null  int16  
 4   fips                  15769 non-null  float64
 5   fips_trimmed          15769 non-null  float64
 6   republican_candidate  15769 non-null  object 
 7   democrat_candidate    15769 non-null  object 
 8   republican_votes      15769 non-null  int64  
 9   democrat_votes        15769 non-null  int64  
 10  county_totalvotes     15769 non-null  float64
 11  winning candidate     15769 non-null  object 
 12  winning party         15769 non-null  int8   
 13  avg_wage_salary       15769 non-null  int64  
dtypes: float64(3), int16(1), int64(4), int8(3), object(3)
memory usage: 1.

In [202]:
## create dummies to get rid of string values.
# winning_party_binary = pd.get_dummies(data['winning party'], drop_first=True)
# #now republican=1, democrat=0
# data = pd.concat([data, winning_party_binary], axis=1)

In [203]:
# data.head(10)

,year,state,state_po,county,fips,fips_trimmed,republican_candidate,democrat_candidate,republican_votes,democrat_votes,county_totalvotes,winning candidate,winning party,avg_wage_salary
0,2000,1,0,479,2001.0,2001.0,George W. Bush,Al Gore,4681,1284,6656.0,George W. Bush,1,0
1,2000,1,0,490,2002.0,2002.0,George W. Bush,Al Gore,4235,2081,7388.0,George W. Bush,1,0
2,2000,1,0,501,2003.0,2003.0,George W. Bush,Al Gore,3135,3693,8225.0,Al Gore,0,0
3,2000,1,0,512,2004.0,2004.0,George W. Bush,Al Gore,4127,2715,7823.0,George W. Bush,1,0
4,2000,1,0,514,2005.0,2005.0,George W. Bush,Al Gore,3545,1931,6437.0,George W. Bush,1,0
5,2000,1,0,515,2006.0,2006.0,George W. Bush,Al Gore,3862,1542,6100.0,George W. Bush,1,0
6,2000,1,0,516,2007.0,2007.0,George W. Bush,Al Gore,4868,1893,8677.0,George W. Bush,1,0
7,2000,1,0,517,2008.0,2008.0,George W. Bush,Al Gore,5371,1498,7978.0,George W. Bush,1,0
8,2000,1,0,518,2009.0,2009.0,George W. Bush,Al Gore,4789,1203,6800.0,George W. Bush,1,0
9,2000,1,0,480,2010.0,2010.0,George W. Bush,Al Gore,5673,2194,8884.0,George W. Bush,1,0


In [71]:
data.drop(['winning candidate', 'republican_candidate', 'democrat_candidate', 'winning party', 'CTYNAME', 'STNAME', 'AGEGRP'], axis=1, inplace=True)

In [72]:
from sklearn.model_selection import train_test_split

In [73]:
X=data.drop("winning_party_binary", axis=1)
y=data["winning_party_binary"]

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=11)

In [75]:
from sklearn.linear_model import LogisticRegression

In [76]:
model = LogisticRegression()

In [77]:
X_train

,Unnamed: 0,Unnamed: 0.1,ID,FIPS_x,YEAR,STATE,COUNTY,AA_FEMALE,AA_MALE,BA_FEMALE,...,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%,Unemployment_Rate
12219,15564,15564,200456011,56011,2004,56,11,5,0,1,...,0.000000,0.479686,0.502424,0.488714,0.511286,0.269353,0.195787,0.323524,0.211336,3.8
13758,17174,17174,200831057,31057,2008,31,57,3,0,2,...,0.000498,0.493532,0.483085,0.505473,0.494527,0.243781,0.163184,0.309453,0.283582,2.9
10935,14278,14278,200437079,37079,2004,37,79,19,11,3821,...,0.000353,0.284865,0.304111,0.485590,0.514410,0.278466,0.289651,0.270355,0.161528,6.2
3255,3625,3625,201254003,54003,2012,54,3,526,375,3518,...,0.000229,0.452049,0.439086,0.503900,0.496100,0.273263,0.261572,0.291150,0.174015,6.7
1301,1670,1670,201221091,21091,2012,21,91,14,7,29,...,0.000000,0.483452,0.492691,0.493743,0.506257,0.281604,0.225588,0.287686,0.205122,7.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4023,4394,4394,201620097,20097,2016,20,97,22,12,9,...,0.000000,0.481043,0.473144,0.504344,0.495656,0.258689,0.228673,0.250395,0.262243,2.8
7259,10528,10528,200027017,27017,2000,27,17,62,52,21,...,0.000063,0.460306,0.460148,0.493171,0.506829,0.279767,0.248100,0.279104,0.193029,4.9
5200,5571,5571,201639141,39141,2016,39,141,207,168,1071,...,0.000104,0.445786,0.463311,0.475517,0.524483,0.238374,0.249141,0.298756,0.213729,5.5
3775,4146,4146,201617187,17187,2016,17,187,168,264,209,...,0.000847,0.474857,0.450477,0.507139,0.492861,0.266099,0.239291,0.249506,0.245104,5.3


In [78]:
y

0        1
1        1
2        0
3        1
4        1
        ..
15432    0
15433    0
15434    0
15435    0
15436    0
Name: winning_party_binary, Length: 15437, dtype: int64

In [79]:
model.fit(X_train, y_train)

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [80]:
predictions = model.predict(X_test)

In [81]:
from sklearn.metrics import accuracy_score

In [82]:
accuracy_score(y_test, predictions)

0.8661485319516408

In [83]:
from sklearn.metrics import confusion_matrix

In [84]:
confusion_matrix(y_test, predictions)

array([[ 315,  604],
       [  16, 3697]])